In [8]:
import os
from dotenv import load_dotenv, find_dotenv
from getpass import getpass

# 1. Try to find and load .env file
env_path = find_dotenv()
if env_path:
    load_dotenv(env_path)
    print(f"Loaded environment variables from: {env_path}")
else:
    print("⚠️  No .env file found — falling back to manual input.")

# 2. Helper function to fetch or prompt
def get_env_var(key, prompt_text=None, secret=False):
    """Return environment variable or prompt user if missing."""
    value = os.getenv(key)
    if not value:
        if secret:
            value = getpass(prompt_text or f"Enter {key}: ")
        else:
            value = input(prompt_text or f"Enter {key}: ")
        os.environ[key] = value  # optionally keep it in memory
    return value

# 3. Use it
OPENAI_API_KEY = get_env_var("OPENAI_API_KEY", "OPENAI_API_KEY: ", secret=True)
LANGSMITH_API_KEY = get_env_var("LANGSMITH_API_KEY", "LANGSMITH_API_KEY: ", secret=True)
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "true")

Loaded environment variables from: /Users/jackliu2006/workspace/hello-agent/.env


In [9]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "cv.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

3
Xiao  Liu,  Cloud  Architect/Engineer  
Filderhauptstr,  51,  Stuttgart,  70599,  Germany,  015561665338,  jackliu2006@gmail.com  
  SUMMARY                      
CERTIFICATES    
Sep  2025     Certif

{'producer': 'Skia/PDF m142 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Xiao Liu_Cloud_Arch_EN', 'source': 'cv.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [22]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://genai-nexus.int.api.corpinter.net/apikey/",
    api_version="2024-06-01",
    api_key=OPENAI_API_KEY,
    model="text-embedding-3-large"
)

In [23]:
from langchain_milvus import Milvus

URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

In [24]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

['4d309db7-574b-47f0-bbb1-f41e5565809c',
 '7b33558f-c28e-4917-87de-551e9523ef84',
 '331d3e8d-7dee-41e2-ae5e-031420ee88ec']